In [2]:
import pandas as pd
import numpy as np
import re
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [3]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']
    
invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt', header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

member_info = pd.read_table(os.path.join(DATA_PATH, 'member_info_0926.txt', header=None)
member_info.columns = ['用户ID','性别','创作关键词的编码序列','创作数量等级','创作热度等级','注册类型','注册平台','访问频率','用户二分类特征A','用户二分类特征B','用户二分类特征C','用户二分类特征D','用户二分类特征E','用户分类特征A','用户分类特征B','用户分类特征C','用户分类特征D','用户分类特征E','用户的盐值分数','用户关注的话题','用户感兴趣的话题']

oversample = True

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
                            
data = data.merge(member_info, on='用户ID', how='left').merge(question_info, on='问题ID', how='left')

print("Used time: %d s" % (time.time()-tic))

Used time: 106 s


In [4]:
tic = time.time()
tmp = data.groupby(['问题ID'])['用户ID'].count().reset_index()
tmp.columns =['问题ID','问题邀请用户_counts']
data = data.merge(tmp, on='问题ID', how='left')

tmp = data.groupby(['用户ID'])['问题ID'].count().reset_index()
tmp.columns =['用户ID','用户被邀请问题_counts']
data = data.merge(tmp, on='用户ID', how='left')
tmp = data.groupby(['问题ID'])['用户的盐值分数'].agg({'用户的盐值分数_mean':'mean','用户的盐值分数_max':'max','用户的盐值分数_min':'min','用户的盐值分数_std':'std'}).reset_index()
data = data.merge(tmp, on='问题ID', how='left')
    
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
    
tmp = data['问题创建时间'].apply(lambda x : x.split('-'))
data['问题创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['问题创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
    
data['邀请问题创建时间_gap'] = (data['邀请创建时间_D']*24+data['邀请创建时间_H']) - (data['问题创建时间_D']*24 + data['问题创建时间_H'])
tmp = data[['问题邀请用户_counts','用户被邀请问题_counts','用户的盐值分数_mean','用户的盐值分数_max','用户的盐值分数_min','用户的盐值分数_std','邀请问题创建时间_gap','问题创建时间_H','问题创建时间_D','邀请创建时间_D','邀请创建时间_H']]

print("Used time: %d s" % (time.time()-tic))

Used time: 268 s


In [6]:
tmp.to_pickle(os.path.join(FEAT_PATH, 'strong_feat_collections.pickle'))
print("Feature Saved, shape:",tmp.shape)

Feature Saved, shape: (11772563, 11)
